# ☁️ Cloud & DevOps Infrastructure Guide

**Comprehensive guide for mastering cloud platforms and DevOps practices**

---

## 📚 Table of Contents

1. [Cloud Platforms Overview](#1-cloud-platforms-overview)
2. [AWS Services](#2-aws-services)
3. [Azure Services](#3-azure-services)
4. [Docker & Kubernetes](#4-docker--kubernetes)
5. [CI/CD Pipelines](#5-cicd-pipelines)
6. [Infrastructure as Code](#6-infrastructure-as-code)
7. [Monitoring & Logging](#7-monitoring--logging)
8. [Security & Best Practices](#8-security--best-practices)

---

## 1. Cloud Platforms Overview

### Major Cloud Providers

| Provider | Market Share | Key Strengths |
|----------|-------------|---------------|
| **AWS** | ~32% | Most mature, largest service catalog |
| **Azure** | ~23% | Best for Microsoft ecosystem |
| **GCP** | ~10% | Best for data analytics, ML/AI |
| **Others** | ~35% | Alibaba, Oracle, IBM |

### Cloud Service Models

- **IaaS**: Virtual Machines, Storage, Networks
- **PaaS**: App hosting, Databases, Development tools
- **SaaS**: Complete applications (Gmail, Salesforce)

### Deployment Models

- **Public Cloud**: AWS, Azure, GCP
- **Private Cloud**: On-premises dedicated resources
- **Hybrid Cloud**: Mix of public and private
- **Multi-Cloud**: Using multiple cloud providers

## 2. AWS Services

### Core Compute

#### EC2 (Elastic Compute Cloud)
- Virtual servers in the cloud
- Instance types: t2/t3 (general), c5 (compute), r5 (memory), p3 (GPU)
- Auto Scaling Groups for dynamic scaling
- Elastic Load Balancing (ALB, NLB)

#### Lambda (Serverless)
- Event-driven functions
- Supports Python, Node.js, Java, Go
- Pay per invocation

#### ECS/EKS (Containers)
- **ECS**: AWS-native container service
- **EKS**: Managed Kubernetes

### Storage

#### S3 (Simple Storage Service)
- Object storage with unlimited capacity
- Storage classes: Standard, IA, Glacier, Deep Archive
- Lifecycle policies for cost optimization

#### EBS (Elastic Block Store)
- Block storage for EC2 instances
- Volume types: gp3 (SSD), io2 (provisioned IOPS), st1 (HDD)

### Databases

- **RDS**: Managed MySQL, PostgreSQL, Oracle, SQL Server
- **Aurora**: High-performance MySQL/PostgreSQL
- **DynamoDB**: NoSQL key-value database
- **ElastiCache**: Redis and Memcached

### Networking

#### VPC (Virtual Private Cloud)
- Isolated network environment
- Subnets (public/private)
- Security Groups and NACLs
- NAT Gateway for outbound traffic

#### Route 53
- DNS and domain registration
- Health checks and failover

#### CloudFront
- Global CDN for content delivery

## 3. Azure Services

### Compute
- **Virtual Machines**: Windows/Linux VMs
- **App Service**: PaaS for web apps
- **Azure Functions**: Serverless compute
- **AKS**: Managed Kubernetes

### Storage
- **Blob Storage**: Object storage (hot, cool, archive)
- **Azure Files**: Managed file shares
- **Disk Storage**: Managed disks for VMs

### Databases
- **Azure SQL Database**: Managed SQL Server
- **Cosmos DB**: Multi-model NoSQL (globally distributed)
- **Azure Database for MySQL/PostgreSQL**

### Networking
- **Virtual Network (VNet)**: Network isolation
- **Application Gateway**: Layer 7 load balancer
- **Azure Front Door**: Global CDN and load balancer

## 4. Docker & Kubernetes

### Docker

#### Dockerfile Example
```dockerfile
FROM python:3.11-slim
WORKDIR /app
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt
COPY . .
EXPOSE 8000
CMD ["uvicorn", "main:app", "--host", "0.0.0.0"]
```

#### Docker Compose
```yaml
version: '3.8'
services:
  web:
    build: .
    ports:
      - "8000:8000"
    environment:
      - DATABASE_URL=postgresql://user:pass@db:5432/mydb
    depends_on:
      - db
  
  db:
    image: postgres:15
    environment:
      POSTGRES_USER: user
      POSTGRES_PASSWORD: pass
    volumes:
      - postgres-data:/var/lib/postgresql/data

volumes:
  postgres-data:
```

### Kubernetes

#### Deployment
```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: myapp
spec:
  replicas: 3
  selector:
    matchLabels:
      app: myapp
  template:
    metadata:
      labels:
        app: myapp
    spec:
      containers:
      - name: myapp
        image: myapp:v1
        ports:
        - containerPort: 8000
        resources:
          requests:
            memory: "64Mi"
            cpu: "250m"
          limits:
            memory: "128Mi"
            cpu: "500m"
```

#### Service
```yaml
apiVersion: v1
kind: Service
metadata:
  name: myapp-service
spec:
  type: LoadBalancer
  selector:
    app: myapp
  ports:
  - port: 80
    targetPort: 8000
```

## 5. CI/CD Pipelines

### GitHub Actions

```yaml
name: CI/CD

on:
  push:
    branches: [main]

jobs:
  build:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      
      - name: Build Docker image
        run: docker build -t myapp:${{ github.sha }} .
      
      - name: Push to registry
        run: |
          docker login -u ${{ secrets.DOCKER_USER }} -p ${{ secrets.DOCKER_PASS }}
          docker push myapp:${{ github.sha }}
      
      - name: Deploy to Kubernetes
        run: |
          kubectl set image deployment/myapp myapp=myapp:${{ github.sha }}
```

### GitLab CI/CD

```yaml
stages:
  - test
  - build
  - deploy

test:
  stage: test
  script:
    - pytest

build:
  stage: build
  script:
    - docker build -t $CI_REGISTRY_IMAGE:$CI_COMMIT_SHA .
    - docker push $CI_REGISTRY_IMAGE:$CI_COMMIT_SHA

deploy:
  stage: deploy
  script:
    - kubectl set image deployment/myapp myapp=$CI_REGISTRY_IMAGE:$CI_COMMIT_SHA
  only:
    - main
```

## 6. Infrastructure as Code

### Terraform Example

```hcl
# provider.tf
provider "aws" {
  region = "us-east-1"
}

# main.tf
resource "aws_vpc" "main" {
  cidr_block = "10.0.0.0/16"
  
  tags = {
    Name = "main-vpc"
  }
}

resource "aws_subnet" "public" {
  vpc_id            = aws_vpc.main.id
  cidr_block        = "10.0.1.0/24"
  availability_zone = "us-east-1a"
  
  tags = {
    Name = "public-subnet"
  }
}

resource "aws_instance" "web" {
  ami           = "ami-0c55b159cbfafe1f0"
  instance_type = "t3.micro"
  subnet_id     = aws_subnet.public.id
  
  tags = {
    Name = "web-server"
  }
}
```

### Ansible Playbook

```yaml
---
- name: Configure web servers
  hosts: web
  become: yes
  
  tasks:
    - name: Install nginx
      apt:
        name: nginx
        state: present
        update_cache: yes
    
    - name: Start nginx
      service:
        name: nginx
        state: started
        enabled: yes
    
    - name: Deploy application
      copy:
        src: /local/app
        dest: /var/www/html
```

## 7. Monitoring & Logging

### Prometheus

```yaml
global:
  scrape_interval: 15s

scrape_configs:
  - job_name: 'prometheus'
    static_configs:
      - targets: ['localhost:9090']
  
  - job_name: 'node-exporter'
    static_configs:
      - targets: ['localhost:9100']
  
  - job_name: 'application'
    kubernetes_sd_configs:
      - role: pod
```

### ELK Stack (Elasticsearch, Logstash, Kibana)

```ruby
# Logstash configuration
input {
  beats {
    port => 5044
  }
}

filter {
  grok {
    match => { "message" => "%{COMBINEDAPACHELOG}" }
  }
}

output {
  elasticsearch {
    hosts => ["elasticsearch:9200"]
    index => "logs-%{+YYYY.MM.dd}"
  }
}
```

### CloudWatch (AWS)
- Metrics: CPU, memory, disk, network
- Logs: Application and system logs
- Alarms: Notifications based on thresholds
- Dashboards: Visualization

## 8. Security & Best Practices

### Container Security
- Use minimal base images (alpine, distroless)
- Run as non-root user
- Scan images for vulnerabilities (Trivy, Snyk)
- Don't store secrets in images

### Kubernetes Security
- Enable RBAC (Role-Based Access Control)
- Use Network Policies
- Pod Security Standards
- Secrets encryption at rest

### Cloud Security
- Principle of least privilege (IAM policies)
- Enable MFA
- Encrypt data at rest and in transit
- Use VPC for network isolation
- Enable logging and monitoring
- Regular security audits

### Cost Optimization
- Right-size instances
- Use Reserved Instances/Savings Plans
- Auto Scaling
- Delete unused resources
- Monitor with Cost Explorer
- Use storage lifecycle policies